In [159]:
import pandas as pd
import psycopg2 

url = "https://shelterdata.s3.amazonaws.com/shelter1000_new.csv"

df = pd.read_csv(url)

In [160]:
#check if animal_id is valid values
assert df['animal_id'].isnull().sum() == 0

# splitting sex into sex and sex_status by the space
df[['sex_status', 'sex']] = df['sex'].str.split(' ', expand=True)

# check if any names start with '*'
assert len(df[df['name'].str[0] == '*']) == 0

# convert ts to datetime
df['ts'] = pd.to_datetime(df['ts'])

# delete month_year column
del df['month_year']

# convert dob to date
df['dob'] = pd.to_datetime(df['dob'])

# see if values in outcome are valid
assert len(df['outcome'].unique()) == 7

# checking if values in type are as expected
assert len(df['type'].unique()) == 4

# convert all nulls to empty string
df = df.fillna('')



/var/folders/zd/n80lckcd41730p4lc48ysgjm0000gn/T/ipykernel_2270/2925947852.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['ts'] = pd.to_datetime(df['ts'])


In [164]:
# creating outcome_dim table
outcome_dim = df.outcome.value_counts().reset_index().reset_index().loc[:, ['index', 'outcome']].rename(columns={'index': 'outcome_type_id', 'outcome': 'outcome_type'})

# creating sex_status_dim table
sex_status_dim = df.sex_status.value_counts().reset_index().reset_index().loc[:,['index','sex_status']]
sex_status_dim.columns = ['sex_status_id', 'sex_status']

#creating date_dim table
date_dim = df.ts.dt.date.value_counts().reset_index().loc[:,['ts']]
date_dim.columns = ['date_id']
date_dim[['outcome_year','outcome_month','outcome_day']] = date_dim['date_id'].astype(str).str.split('-', expand=True)
date_dim['date_id'] = date_dim['outcome_year'] + date_dim['outcome_month'] + date_dim['outcome_day']
date_dim.date_id = date_dim.date_id.astype('int')

#creating animal_dim table
temp = df.sort_values(by=['ts']).drop_duplicates(subset=['animal_id'], keep='last')

animal_dim = df.animal_id.value_counts().reset_index().loc[:,['animal_id']]
animal_dim = pd.merge(animal_dim, temp[['animal_id','name', 'dob','type','sex','breed','color']], on='animal_id', how='left')
animal_dim

# #creating outcome_fact table
outcomes_fact = df.loc[:,['animal_id','ts','outcome', 'sex_status']]
outcomes_fact.ts = outcomes_fact.ts.dt.date.astype(str).str.replace('-','').astype(int)
outcomes_fact = pd.merge(outcomes_fact, outcome_dim, left_on = 'outcome', right_on = 'outcome_type', how='left')
outcomes_fact = pd.merge(outcomes_fact, sex_status_dim, on='sex_status', how='left')
outcomes_fact.drop(columns= ['outcome_type', 'outcome','sex_status'], inplace=True)
outcomes_fact.rename(columns={'ts':'date_id'}, inplace=True)


,animal_id,date_id,outcome_type_id,sex_status_id
0,A794011,20190508,5,0
1,A776359,20180718,0,0
2,A821648,20200816,3,3
3,A720371,20160213,0,0
4,A674754,20140318,1,2
...,...,...,...,...
995,A691208,20141111,0,1
996,A718620,20160301,0,1
997,A705114,20150613,1,2
998,A800888,20190801,1,2


In [170]:
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    database="shelter_db",
    user="ujas",
    password="postgres_pass"
)

In [167]:
cur = conn.cursor()

for index, row in outcome_dim.iterrows():
    cur.execute(
        "INSERT INTO outcome_dim (outcome_type_id, outcome_type) VALUES (%s, %s)",
        (row['outcome_type_id'], row['outcome_type']))

for index, row in sex_status_dim.iterrows():
    cur.execute(
        "INSERT INTO sex_status_dim (sex_status_id, sex_status) VALUES (%s, %s)",
        (row['sex_status_id'], row['sex_status']))
    
for index, row in date_dim.iterrows():
    cur.execute(
        "INSERT INTO date_dim (date_id, outcome_year, outcome_month, outcome_day) VALUES (%s, %s, %s, %s)",
        (row['date_id'], row['outcome_year'], row['outcome_month'], row['outcome_day']))
    
for index, row in animal_dim.iterrows():
    cur.execute(
        "INSERT INTO animal_dim (animal_id, animal_name, animal_dob, animal_type, animal_sex, animal_breed, animal_color) VALUES (%s, %s, %s, %s, %s, %s, %s)",
        (row['animal_id'], row['name'], row['dob'], row['type'], row['sex'], row['breed'], row['color']))
    
for index, row in outcomes_fact.iterrows():
    cur.execute(
        "INSERT INTO outcomes_fact (animal_id, date_id, outcome_type_id, sex_status_id) VALUES (%s, %s, %s, %s)",
        (row['animal_id'], row['date_id'], row['outcome_type_id'], row['sex_status_id']))

    
conn.commit()
cur.close()
conn.close()


In [171]:
# cur = conn.cursor()

# cur.execute('select * from outcomes_fact limit 10;')
# results = cur.fetchall()
# for row in results:
#     print(row)

# cur.close()
# conn.close()


(1, 'A794011', 20190508, 5, 0)
(2, 'A776359', 20180718, 0, 0)
(3, 'A821648', 20200816, 3, 3)
(4, 'A720371', 20160213, 0, 0)
(5, 'A674754', 20140318, 1, 2)
(6, 'A659412', 20201005, 0, 1)
(7, 'A814515', 20200506, 0, 0)
(8, 'A860161', 20220719, 0, 0)
(9, 'A689724', 20141018, 0, 0)
(10, 'A680969', 20140805, 0, 0)
